<a href="https://colab.research.google.com/github/andresni/miniPHI/blob/master/miniPHI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Approximations and heuristics of PHI
This is a notebook for generating logic gate networks, calculating PHI on them, as well as approximatinos and heuristics to PHI.

PHI is a measure of integrated information, from the Integrated Information Theory of Consciousness (IIT) by Guilio Tononi (Oizumi et al. 2014).

Here we will generate many many networks, and see if there are metrics that are less resource intensive than PHI, but perhaps accurate.

See the paper  [Nilsen, Juel, and Marshall (2019)](https://www.mdpi.com/1099-4300/21/5/525) for more information.

---
First, installs and imports.

In [0]:
%pip install pyphi

import numpy as np
import pandas as pd
import copy
import time
import itertools
import os
import csv
import random
import scipy.io as sio
import scipy as spi

Next is parameters. 

"Steps" which refers to how many states should a network visit following a given initialization. This will be normalized if multiple network sizes are used so that the information content of different network sizes are the same (i.e. fewer nodes = more steps).

"Samples" refers to how many times should a given initialization be repeated.

"States" refers to how many initializations should be performed

"noiseLVL" refers to how much noise should be present in the network activity.

"noise" refers to type of noise, with options "con" (fuzzy connections), "node" (fuzzy nodes), "net" (embedded network, i.e. hidden background nodes influencing network).

In [0]:
# Setting up parameters
Params = {"nrnetworks": 5,  # generate how many networks of each size
          "minsize": 3,  # min node count
          "maxsize": 6,  # max node count
          "steps": 32,  # number of steps per network
          "samples": 1,  # number of samples per state
          "states": 3200,  # number of states to start in
          "noiseLVL": 0.00,  # plus minus noise
          "noise": "none",  # kind of noise (none, con, node, net)
          }

Next is a function to generate networks $M$ of the following kind:

$M_{i,j} =
 \begin{pmatrix}
  0 & w_{1,2} & \cdots & w_{1,j} \\
  w_{2,1} & 0 & \cdots & w_{2,j} \\
  \vdots  & \vdots  & \ddots & \vdots  \\
  w_{m,1} & w_{i,2} & \cdots & 0
 \end{pmatrix}$
 
 where $w$ is the weight between nodes $i$ and $j$.

In [0]:
def net_gen_next(Params):
    networks = []
            
    for size in range(Params["minsize"], Params["maxsize"] + 1):
        temp_net = []
        for x in range(0, Params["nrnetworks"]):
            temp = np.zeros([size,size])
            
            conn_prob = np.random.uniform(0.2, 1.0)
            inh_prob = np.random.uniform(0.0, 0.8)
            
            for xx in range(0,size):
                for yy in range(0,size):
                    if not xx==yy:
                        if np.random.rand() < conn_prob:
                            temp[xx][yy] = np.random.normal(1, 0)
                            if np.random.rand() < inh_prob:
                                temp[xx][yy] = np.random.normal(-1, 0)
            
            temp2 = list(itertools.permutations([it for it in range(size)]))
            temp3 = [temp[:, list(it)][list(it)] for it in temp2]
            
            yes = 0
            for ii in temp3:
                for zz in temp_net:    
                    if len(np.unique([ii, zz], axis=0)) < 2:
                        yes = 1
                        break
            if not yes:
              temp_net.append(copy.copy(temp))
                    
        for xx in temp_net:
            networks.append(xx)
    
    return np.array(networks)

And then functions related to generate activity, along with some auxillary functions.

In [0]:
def createLOLI(nodes): # Creates an ordered list that iterates through all possible permutations of a binary array.
    LOLI = np.zeros((int(2**nodes), int(nodes)))
    i = 1
    for z in range(0, len(LOLI[0, :])):
        for x in range(i, len(LOLI[:, 0])):
            if LOLI[x - i, z] == 1:
                LOLI[x, z] = 0
            else:
                LOLI[x, z] = 1
        i = i * 2       
    return LOLI

def makeStateByNode(spm,LOLI): # Convert activity to state by node probability matrix
         
    sbn=np.zeros((len(LOLI),len(LOLI[0])+1))
    LOLI=LOLI.tolist()
    a = int(len(spm[0][0]) - len(LOLI[0]))
    for x in range(len(spm)):
      for i in range(len(spm[x])-1):
        b=spm[x][i][:-a] if a > 0 else spm[x][i]
        c=spm[x][i+1][:-a] if a > 0 else spm[x][i+1]
        indexa=LOLI.index(list(b))
        c=np.append(c,1)
        sbn[indexa]=np.add(sbn[indexa],c)
        
    for i in range(0,len(sbn)):
        for q in range(0,len(sbn[i])-1):
            sbn[i,q]=sbn[i,q]/sbn[i][-1] if sbn[i][-1] > 0 else 0
            sbn[i,q]=np.round(sbn[i,q],4)
    
    return sbn[:,0:-1]

def makeStateByState(spm,LOLI): # Convert activity to state by state probability matrix
    
    sbn=np.zeros((len(LOLI),len(LOLI)))
    a = int(len(spm[0][0]) - len(LOLI[0]))
    LOLI=LOLI.tolist()
    for x in range(len(spm)):
      for i in range(len(spm[x])-1):  
        b=spm[x][i][:-a] if a > 0 else spm[x][i]
        c=spm[x][i+1][:-a] if a > 0 else spm[x][i+1]
        indexa=LOLI.index(list(b))
        indexb=LOLI.index(list(c))
        sbn[indexa,indexb]+=1
        
    for i in range(0,len(sbn)):
        s=np.sum(sbn[i])
        for q in range(0,len(sbn[i])):
            sbn[i,q]=sbn[i,q]/s if s > 0 else 0
            sbn[i,q]=np.round(sbn[i,q],4)
    
    return sbn

def gen_transition_prob_matrices_state(results): # Create transition probability matrix (state by state)
   
    tpm = []    
    for i in range(len(results["partial_activity"])):
        tpm.append(copy.deepcopy(statsPlots.makeStateByState(results["partial_activity"][i],results["LOLI"][i])))
    results.update({"tpms":copy.deepcopy(tpm)})
    return results

def gen_transition_prob_matrices_node(results): # Create transition probability matrix (state by node)
    
    tpm = []
    for i in range(len(results["partial_activity"])):
        tpm.append(copy.deepcopy(statsPlots.makeStateByNode(results["partial_activity"][i],results["LOLI"][i])))
    results.update({"tpmn":copy.deepcopy(tpm)})
    return results

def activity(results,Params): # Generate activity given a network
    
    all_net_act = []
    all_net_pact = []
    all_net_LOLI = []
    
    for i in results["networks"]:
        partial = []
        LOLI = createLOLI(len(i))
        replace = True if Params["states"] > len(LOLI) else False
        states = np.random.choice(list(range(len(LOLI))), Params["states"], replace = replace)
        states = list(states) * Params["steps"]

        for L in states:
            state_block = LOLI[L]
            for rounds in range(Params["samples"]):
                temp = np.zeros(len(i))
                for x in range(len(i)):
                    if LOLI[L][x] > 0:
                        for y in range(len(i)):
                          noise = np.random.uniform(-Params["noiseLVL"], Params["noiseLVL"]) if Params["noise"] == "con" else 0
                          temp[y] = temp[y] + i[x][y] + np.random.uniform(noise, noise)
                for x in range(len(i)):
                  temp[x] = 1 if temp[x] >= 1 else 0
                  if Params["noise"] == "node":
                    temp[x] = [1, 0][int(temp[x])] if np.random.rand() < Params["noiseLVL"] else [0, 1][int(temp[x])]
                state_block.append(copy.deepcopy(temp))
            partial.append(copy.deepcopy(state_block))

        all_net_pact.append(copy.deepcopy(partial))
        all_net_LOLI.append(copy.deepcopy(LOLI))

    return {"partial_activity": all_net_pact, "LOLI": all_net_LOLI}